In [2]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')

In [3]:
df = pd.DataFrame(columns = ['Name', 'Total Bedroom', 'Total Bathroom', 'Apart Size', 'Max Capacity', 'Max Watt', 'Address', 'Swim Pool', 'Rating', 'Total Review', 'Furnish Type', 'Price'])
idx = 1

In [85]:
from selenium.webdriver.common.keys import Keys

def get_apartment(num_jobs):
    # driver.get("https://www.travelio.com/search?searchType=monthly&destinationCategory=City&destinationUrlName=&destinationPlaceId=&destinationCountryId=ID&destinationId=544a05f190e50d6a3d000001&nights=31&flexible=1&destination=Jakarta&checkIn=13-08-2021&checkOut=13-09-2021&months=1&cbFlexible=on&unitType=3%2C2%2C1%2Cstudio&propTypeId=&sellType=&bottomPrice=1280000&upperPrice=104400000&hotelName=&areaId=&buildingId=&facilityId=&badge=&instant=&provider=all&sortBy=&sortOrder=")
    # time.sleep(3)
    # driver.maximize_window()
    
    body = driver.find_element_by_tag_name('body')
    global idx
    driver.implicitly_wait(10)

    while True:
        # if(idx > num_jobs):
        #     break
    
        # Total number of apartment loaded in the website
        time.sleep(2)
        apartments = driver.find_elements_by_xpath('//div[@id="property-box-wrapper"]//div[@class="property-outer-box col-xs-12  completed"]')
        length = len(apartments)
  
        if(idx > length):    
            last_height = driver.execute_script("return document.body.scrollHeight")
            body.send_keys(Keys.END)
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")           
            if(new_height == last_height):
                break
            else:
                last_height = new_height

        apartment = driver.find_element_by_xpath('//div[@id="property-box-wrapper"]//div[@class="property-outer-box col-xs-12  completed"]['+str(idx)+']')
        apartmentID = apartment.get_attribute('id')
        print("test")
        # Obtain apartment details: total bedroom, total bathroom, total size, total rating, total furnish_type
        name = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-name"]').text
        no_bedroom = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-info"]//span[2]').text
        no_bathroom = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-info"]//span[3]').text
        size = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-info"]//span[4]').text
        rating = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-info"]//span[5]').text
        furnish_type= driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="property-detail-info"]//span[5]').text
        price = driver.find_element_by_xpath('//*[@id="' + apartmentID +'"]//div[@class="price"]').text

        print(f"Apartment #{idx}")
        # print("Name: {}".format(name))
        # print("Total Bedroom: {}".format(no_bedroom))
        # print("Total Bathroom : {}".format(no_bathroom))
        # print("Apart Size: {}".format(size))
        # print("Rating: {}".format(rating))
        # print("Furnish Type: {}".format(furnish_type))
        # print("Price: {}".format(price))

        # Switch to New tab
        apartment.click()
        driver.switch_to.window(driver.window_handles[1])
        
        # CLose the Notification Pop-up
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="close-button"]'))         
            )
            element.click()
        except:
            break

                
        time.sleep(2)
        # Obtain the total number of review
        if(rating != 'New'):
            try:
                no_review = driver.find_element_by_xpath('//a[@id="property-rate-base"]//span').text
            except NoSuchElementException:
                no_review = -1   
        else:
            no_review = -1

        # Obtain the Apartment Address

        try:
            address = WebDriverWait(driver, 7).until(
                EC.presence_of_element_located((By.XPATH, '//div[@id="hotel-address"]//span[@itemprop="address"]'))   
            )
            # address = driver.find_element_by_xpath('//div[@id="hotel-address"]//span[@itemprop="address"]').text   
        except NoSuchElementException:
            address = -1
        try:
            max_capacity = driver.find_element_by_xpath('//div[@id="room-capacity-info"]').text
        except NoSuchElementException:
            max_capacity = -1
        try:
            max_watt = driver.find_element_by_xpath('(//div[@id="room-capacity-info"])[2]').text
        except NoSuchElementException:
            max_watt = -1
        try:
           driver.find_element_by_xpath("//div[@class='hotel-info-facility-name' and ./text()='Swimming Pool']").text
           swim_pool = 1
        except NoSuchElementException:
            swim_pool = -1

        # print("Total Review: {}".format(no_review))
        # print("Address: {}".format(address))
        # print("Max_capacity: {}".format(max_capacity))
        # print("Max_Watt: {}".format(max_watt))
        # print("Swim Pool: {}".format(swim_pool))
        # print("\n")
        
        # Switch to new tab
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        new_row = { "Name" : name,
                    "Total Bedroom" : no_bedroom,
                    "Total Bathroom" : no_bathroom,
                    "Apart Size"  : size,
                    "Max Capacity" : max_capacity,
                    "Max Watt" : max_watt,
                    "Address" : address,
                    "Swim Pool" : swim_pool,
                    "Rating" : rating,
                    "Total Review" : no_review,
                    "Furnish Type" : furnish_type,
                    "Price" : price}
        
        global df
        idx+=1
        df = df.append(new_row, ignore_index = True)
    print("Number of jobs executed: {}/{}".format(len(df), num_jobs))

In [ ]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
# driver = webdriver.Chrome(PATH)
opt = Options()
opt.add_experimental_option("debuggerAddress", "localhost:8989")
driver = webdriver.Chrome(executable_path=PATH, options=opt)

get_apartment(3000)

# previous_height = driver.execute_script('return document.body.scrollHeight')

# while True:
#     driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

#     time.sleep(3)

#     new_height = driver.execute_script('return document.body.scrollHeight')
#     if new_height == previous_height:
#         break
#     previous_height = new_height

In [102]:
df[df['Address'] == ''].index

Int64Index([39, 83, 130, 155, 324, 327, 331, 335, 337, 348, 357, 361, 366, 440, 446, 544, 545, 590, 591, 593, 619, 624, 625, 626, 635, 636, 637, 722, 740, 742, 745, 747, 772], dtype='int64')

In [94]:
df.iloc[335]['Address']

''

In [103]:
# len(df)
print(idx)

64


In [81]:
df.to_csv('travelio.csv', index = True)